In [6]:
import pandas as pd
import numpy as np
import requests 

In [2]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    soupsieve-1.9.2            |           py36_0          61 KB  anaconda
    openssl-1.1.1              |       h7b6447c_0         5.0 MB  anaconda
    beautifulsoup4-4.8.0       |           py36_0         147 KB  anaconda
    certifi-2019.6.16          |           py36_1         156 KB  anaconda
    ------------------------------------------------------------
                                           Total:         5.4 MB

The following NEW packages will be INSTALLED:

    soupsieve:      1.9.2-py36_0      a

In [8]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

In [9]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

In [11]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source)

In [12]:
table = soup.find('table',{'class':'wikitable sortable'})

In [13]:
table_rows = table.find_all('tr')

In [14]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]

In [19]:
df.head(10)

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
10,M8A,Not assigned,Not assigned


In [20]:
df.shape

(288, 3)

In [21]:
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)

In [22]:
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [23]:
df1 = df.reset_index()

In [27]:
df2= df1.groupby('PostalCode').agg(lambda x: ','.join(x))

In [28]:
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [29]:
df2.loc[df2['Neighbourhood']=="Not assigned",'Neighbourhood']=df2.loc[df2['Neighbourhood']=="Not assigned",'Borough']

In [30]:
df3 = df2.reset_index()

In [31]:
df3['Borough']= df3['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [32]:
df3.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [33]:
df3.shape

(103, 3)

# question 2

In [36]:
pip install geopy

     |████████████████████████████████| 102kB 996kB/s ta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/99/45/d1/14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib
Note: you may need to restart the kernel to use updated packages.


In [37]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
location = geolocator.geocode("Toronto, North York, Parkwoods")

print(location.address)
print('')
print((location.latitude, location.longitude))
print('')
print(location.raw)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Ontario, M3A 2X2, Canada

(43.7587999, -79.3201966)

{'place_id': 112665056, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 160406961, 'boundingbox': ['43.7576231', '43.761106', '-79.3239088', '-79.316215'], 'lat': '43.7587999', 'lon': '-79.3201966', 'display_name': 'Parkwoods Village Drive, Parkway East, Don Valley East, North York, Toronto, Ontario, M3A 2X2, Canada', 'class': 'highway', 'type': 'secondary', 'importance': 0.51}


In [40]:
df_geopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})
from geopy.geocoders import Nominatim
geolocator = Nominatim()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


In [41]:
df_geopy1 = df3

In [42]:
from geopy.geocoders import Nominatim
geolocator = Nominatim()

df_geopy1['address'] = df3[['PostalCode', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
df_geopy1.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


,PostalCode,Borough,Neighbourhood,address
0,M1B,Scarborough,"Rouge,Malvern","M1B, Scarborough, Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C, Scarborough, Highland Creek,Rouge Hill,Po..."
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E, Scarborough, Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn,"M1G, Scarborough, Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H, Scarborough, Cedarbrae"


In [43]:
df_geopy1.drop(df_geopy1[df_geopy1['Borough']=="Notassigned"].index,axis=0, inplace=True)

In [44]:
location = geolocator.geocode("M1G, Scarborough, Woburn")

In [45]:
pip install geocoder

     |████████████████████████████████| 102kB 1.0MB/s ta 0:00:01
     |████████████████████████████████| 829kB 3.1MB/s eta 0:00:01
     |████████████████████████████████| 81kB 11.1MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/0c/61/d2/d6b7317325828fbb39ee6ad559dbe4664d0896da4721bf379e
Successfully built future
Note: you may need to restart the kernel to use updated packages.


In [46]:
df_geopy=df3

In [47]:
import geopy

In [48]:
df_geopy['address']=df_geopy['PostalCode'] + ',' + df_geopy['Borough'] + ','+ df_geopy['Neighbourhood']
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address
0,M1B,Scarborough,"Rouge,Malvern","M1B,Scarborough,Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C,Scarborough,Highland Creek,Rouge Hill,Port..."
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E,Scarborough,Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn"
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae"


In [49]:
nom = Nominatim()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [50]:
n=nom.geocode('M1B, Scarborough, Rouge,Malvern')
n

Location(Malvern, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 4Y7, Canada, (43.8091955, -79.2217008, 0.0))

In [51]:
n.latitude

43.8091955

In [52]:
n2=nom.geocode('M1E Scarborough Guildwood,Morningside,West Hill')
print(n2)

None


In [53]:
df_geopy['Coordinates'] =df_geopy['address'].apply(nom.geocode)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates
0,M1B,Scarborough,"Rouge,Malvern","M1B,Scarborough,Rouge,Malvern","(Malvern, Scarborough—Rouge Park, Scarborough,..."
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C,Scarborough,Highland Creek,Rouge Hill,Port...",None
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E,Scarborough,Guildwood,Morningside,West Hill",None
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T..."
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None


In [54]:
df_geopy.Coordinates[0]

Location(Malvern, Scarborough—Rouge Park, Scarborough, Toronto, Golden Horseshoe, Ontario, M1B 4Y7, Canada, (43.8091955, -79.2217008, 0.0))

In [55]:
df_geopy['latitude']=df_geopy['Coordinates'].apply(lambda x: x.latitude if x !=None else None)
df_geopy['longitude']=df_geopy['Coordinates'].apply(lambda x: x.longitude if x !=None else None)
df_geopy.head()

,PostalCode,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,M1B,Scarborough,"Rouge,Malvern","M1B,Scarborough,Rouge,Malvern","(Malvern, Scarborough—Rouge Park, Scarborough,...",43.809196,-79.221701
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C,Scarborough,Highland Creek,Rouge Hill,Port...",None,NaN,NaN
2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E,Scarborough,Guildwood,Morningside,West Hill",None,NaN,NaN
3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","(Woburn, Scarborough—Guildwood, Scarborough, T...",43.759824,-79.225291
4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",None,NaN,NaN


In [57]:
df_geopy.to_csv('geo_loc_py.csv')

In [58]:
print('The latitude of', df_geopy.address[0],  'is', df_geopy.latitude[0], 'and its longitude is',df_geopy.longitude[0])

The latitude of M1B,Scarborough,Rouge,Malvern is 43.8091955 and its longitude is -79.2217008


In [60]:
df3.to_csv('geopy.csv')

In [61]:
data2 = pd.read_csv("geopy.csv") 
data2.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,address,Coordinates,latitude,longitude
0,0,M1B,Scarborough,"Rouge,Malvern","M1B,Scarborough,Rouge,Malvern","Malvern, Scarborough—Rouge Park, Scarborough, ...",43.809196,-79.221701
1,1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union","M1C,Scarborough,Highland Creek,Rouge Hill,Port...",NaN,NaN,NaN
2,2,M1E,Scarborough,"Guildwood,Morningside,West Hill","M1E,Scarborough,Guildwood,Morningside,West Hill",NaN,NaN,NaN
3,3,M1G,Scarborough,Woburn,"M1G,Scarborough,Woburn","Woburn, Scarborough—Guildwood, Scarborough, To...",43.759824,-79.225291
4,4,M1H,Scarborough,Cedarbrae,"M1H,Scarborough,Cedarbrae",NaN,NaN,NaN
